In [1]:
# 데이터 읽기
import pandas as pd
wine = pd.read_csv('http://bit.ly/wine_csv_data')

In [2]:
# 훈련 데이터와 테스트 데이터 분류
data = wine[['alcohol', 'sugar', 'pH']].to_numpy()
target = wine['class'].to_numpy()

from sklearn.model_selection import train_test_split
train_input, test_input, train_target, test_target = train_test_split(data, target, test_size=0.2, random_state = 42)

In [3]:
# 훈련세트와 검증 세트로 분류
sub_input, val_input, sub_target, val_target = train_test_split(train_input, train_target, test_size = 0.2, random_state = 42)

In [4]:
# 결정 트리 모델 학습
from sklearn.tree import DecisionTreeClassifier
dt = DecisionTreeClassifier(random_state = 42)
dt.fit(sub_input, sub_target)
print(dt.score(sub_input, sub_target))
print(dt.score(val_input, val_target))

0.9971133028626413
0.864423076923077


In [5]:
# 교차 검증
from sklearn.model_selection import cross_validate
scores = cross_validate(dt, train_input, train_target)
print(scores)

{'fit_time': array([0.00996757, 0.00821042, 0.01090598, 0.0085988 , 0.00715613]), 'score_time': array([0.00116229, 0.00178981, 0.00176048, 0.00120926, 0.00108862]), 'test_score': array([0.86923077, 0.84615385, 0.87680462, 0.84889317, 0.83541867])}


In [6]:
# StratifiedKFold 사용하여 교차 검증
from sklearn.model_selection import StratifiedKFold
scores = cross_validate(dt, train_input, train_target,cv = StratifiedKFold())

import numpy as np
print(np.mean(scores['test_score']))

0.855300214703487


In [7]:
# 10폴드 교차검증
splitter = StratifiedKFold(n_splits = 10, shuffle = True, random_state = 42)
scores = cross_validate(dt, train_input, train_target, cv= splitter)
print(np.mean(scores['test_score']))

0.8574181117533719


In [8]:
# 그리드 서치 수행
from sklearn.model_selection import GridSearchCV
params = {'min_impurity_decrease' : [0.0001, 0.0002, 0.0003, 0.0004, 0.0005]}
gs = GridSearchCV(DecisionTreeClassifier(random_state=42), params, n_jobs=-1)

gs.fit(train_input, train_target)
dt = gs.best_estimator_
print('dt.score(train_input, train_target) : ', dt.score(train_input, train_target))
print("best_params_ : ", gs.best_params_)
print('cv_results_[mean_test_score] : ', gs.cv_results_['mean_test_score'])

dt.score(train_input, train_target) :  0.9615162593804117
best_params_ :  {'min_impurity_decrease': 0.0001}
cv_results_[mean_test_score] :  [0.86819297 0.86453617 0.86492226 0.86780891 0.86761605]


In [9]:
# 그리드 서치 수행 (여러 개의 최적의 하이퍼파라미터 탐색)
params = {'min_impurity_decrease' : np.arange(0.0001, 0.001, 0.0001), 'max_depth' : range(5, 20, 1), 'min_samples_split' : range(2, 100, 10)}
gs = GridSearchCV(DecisionTreeClassifier(random_state=42), params, n_jobs=-1)
gs.fit(train_input, train_target)

dt = gs.best_estimator_
print('dt.score(train_input, train_target) : ', dt.score(train_input, train_target))
print("best_params_ : ", gs.best_params_)
print('cv_results_[mean_test_score] : ', gs.cv_results_['mean_test_score'])

dt.score(train_input, train_target) :  0.892053107562055
best_params_ :  {'max_depth': 14, 'min_impurity_decrease': 0.0004, 'min_samples_split': 12}
cv_results_[mean_test_score] :  [0.85780355 0.85799604 0.85799604 ... 0.86126601 0.86165063 0.86357629]


In [10]:
# 랜덤 서치
from scipy.stats import uniform, randint
params = {'min_impurity_decrease' : uniform(0.0001, 0.001), 'max_depth' : randint(20, 50), 'min_samples_split' : randint(2,25), 'min_samples_leaf' : randint(1,25)}

from sklearn.model_selection import RandomizedSearchCV
gs = RandomizedSearchCV(DecisionTreeClassifier(random_state=42), params, n_iter=100, n_jobs=-1, random_state=42)
gs.fit(train_input, train_target)

import numpy as np
dt = gs.best_estimator_
print('dt.score(train_input, train_target) : ', dt.score(train_input, train_target))
print("best_params_ : ", gs.best_params_)
print('np.max(cv_results_[mean_test_score]) : ', np.max(gs.cv_results_['mean_test_score']))

dt.score(train_input, train_target) :  0.8928227823744468
best_params_ :  {'max_depth': 39, 'min_impurity_decrease': 0.00034102546602601173, 'min_samples_leaf': 7, 'min_samples_split': 13}
np.max(cv_results_[mean_test_score]) :  0.8695428296438884
